In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as tts

import tensorflow as tf

In [ ]:
data = pd.read_csv("../input/gender-classification/Transformed Data Set - Sheet1.csv")

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
color_order = list(data['Favorite Color'].unique())

In [ ]:
{column : data[column].unique() for column in data.columns}

In [ ]:
def add_prefixes(df,column,prefix):
    return df[column].apply(lambda x: prefix+x)

In [ ]:
data['Favorite Soft Drink'] = add_prefixes(data,'Favorite Soft Drink','s_')

In [ ]:
data

In [ ]:
data['Favorite Beverage'] = add_prefixes(data,'Favorite Beverage','b_')

In [ ]:
data

In [ ]:
def onehot_encode(df,columns):
    for column in columns:
        dummies = pd.get_dummies(df[column])
        df = pd.concat([df,dummies],axis = 1)
        df.drop(column,axis = 1,inplace = True)
    return df

In [ ]:
data = onehot_encode(data,['Favorite Music Genre','Favorite Beverage','Favorite Soft Drink'])

In [ ]:
data

In [ ]:
data['Favorite Color'] = data['Favorite Color'].apply(lambda x : color_order.index(x))

In [ ]:
data

In [ ]:
encoder_label = LabelEncoder()
data['Gender'] = encoder_label.fit_transform(data['Gender'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
y = data['Gender']
x = data.drop('Gender',axis = 1)

In [ ]:
scaler = MinMaxScaler()
x = pd.DataFrame(scaler.fit_transform(x),columns = x.columns)

In [ ]:
x

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(data.corr(),annot = True,vmin = -1,vmax = 1)

In [ ]:
x_train,x_test,y_train,y_test = tts(x,y,train_size=0.7,random_state=0)

In [ ]:
inputs = tf.keras.Input(shape=(18,))
x = tf.keras.layers.Dense(64,activation='relu')(inputs)
x = tf.keras.layers.Dense(64,activation='relu')(x)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs,outputs=outputs)

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

batch_size = 32
epochs = 24



In [ ]:
history = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    
    batch_size = batch_size,
    epochs = epochs
    
)

In [ ]:
plt.figure(figsize=(14,10))
epochs_range = range(1,epochs+1)
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs_range,val_loss,label = 'Validation Loss')
plt.plot(epochs_range,train_loss,label = 'Loss')

plt.show()

In [ ]:
model.evaluate(x_test,y_test)